In [ ]:
!pip install flask gradio -q
!apt-get install nodejs npm -y
!npm install -g localtunnel

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  gyp javascript-common libc-ares2 libjs-events libjs-highlight.js
  libjs-inherits libjs-is-typedarray libjs-psl libjs-source-map
  libjs-sprintf-js libjs-typedarray-to-buffer libnode-dev libnode72
  libnotify-bin libnotify4 libuv1-dev node-abab node-abbrev node-agent-base
  node-ansi-regex node-ansi-styles node-ansistyles node-aproba node-archy
  node-are-we-there-yet node-argparse node-arrify node-asap node-asynckit
  node-balanced-match node-brace-expansion node-builtins node-cacache
  node-chalk node-chownr node-clean-yaml-object node-cli-table node-clone
  node-color-convert node-color-name node-colors node-columnify
  node-combined-stream node-commander node-console-control-strings
  node-copy-concurrently node-core-util-is node-coveralls node-cssom
  node-cssstyle node-debug node-decompress-response node-defaults
  node-delayed-st

In [ ]:
from flask import Flask, request, jsonify
from datetime import datetime
import threading

app = Flask(__name__)
latest_data = {"distance": None, "status": "Waiting for data...", "timestamp": None}

@app.route('/data', methods=['POST'])
def receive_data():
    global latest_data
    data = request.get_json()
    distance = float(data.get("distance"))

    # 🚘 ML-like Prediction Logic
    if distance > 20:
        status = "Safe ✅"
    elif distance > 10:
        status = "Caution ⚠️"
    else:
        status = "Unsafe 🚨"

    latest_data = {
        "distance": round(distance, 2),
        "status": status,
        "timestamp": datetime.now().strftime("%H:%M:%S")
    }

    print(f"[{latest_data['timestamp']}] {latest_data['distance']} cm → {status}")
    return jsonify({"status": "received"})

@app.route('/live', methods=['GET'])
def get_latest_data():
    return jsonify(latest_data)

# Start server in background
def run_flask():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'
 * Debug mode: off


In [ ]:
import subprocess

def start_localtunnel():
    !pkill node  # Stop existing tunnels
    tunnel = subprocess.Popen(['lt', '--port', '5000'], stdout=subprocess.PIPE)
    for line in tunnel.stdout:
        decoded_line = line.decode()
        if 'https://' in decoded_line:
            print("🚀 Tunnel ready →", decoded_line.strip())
            break

start_localtunnel()